In [1]:
import shutil
import os
import random
import os
import re

def format_line(line):
    line = line.lower()
    line = re.sub(r'\s*,\s*', ' , ', line)
    line = re.sub(r'\s*\.\s*', ' . ', line)
    line = re.sub(r"n \s*'\s*t", "n't", line)
    return line

def split_read(file_name):
    inputs, targets =[], []
    with open(file_name, 'r', encoding='UTF-8') as fp:
        for line in fp:
            input, target = [], []
            input, target = line.strip().split('####')
            if line != '':
                inputs.append(input)
                targets.append(target)
    print('Data read. Total count: ',len(targets))
    return inputs, targets

def merge_save(inputs, targets, file_name, num=1):
    # 각 target 항목을 num 번씩 복사하여 확장
    targets_expanded = [tar for tar in targets for _ in range(num)]

    # 병합
    merged_data = [inp + "####" + tar for inp, tar in zip(inputs, targets_expanded)]
    # 파일로 저장
    with open(file_name, 'w', encoding='UTF-8') as file:
        for line in merged_data:
            file.write(line + '\n')

    # 정보 출력
    print('Input count:', len(inputs))
    print('Expanded target count:', len(targets_expanded))
    print('Merged data count:', len(merged_data))
    print('Data saved. Total count:', len(merged_data))

def dev_sampleing(file_path, file_name, formatted):
    seed = 2024  # seed 값
    train_ratio = 0.9  # train 비율 (90%)
    source_file = os.path.join(file_path,file_name)
    lines = []
    with open(source_file, 'r') as file:
        for line in file:
            if formatted == True:
                formatted_line = format_line(line.strip())
            else:
                formatted_line = line.strip()
            lines.append(formatted_line)
    
    # Shuffle the data
    random.seed(seed)
    random.shuffle(lines)
    
    # Split the data
    split_index = int(len(lines) * train_ratio)
    train_data = lines[:split_index]
    dev_data = lines[split_index:]

    train_file = os.path.join(file_path,f'train.txt')
    dev_file = os.path.join(file_path,f'dev.txt')
    
    # Save the data to the output files
    with open(train_file, 'w') as file:
        file.writelines(line + '\n' for line in train_data)
    
    with open(dev_file, 'w') as file:
        file.writelines(line + '\n' for line in dev_data)
    
    print(f"Train data count: {len(train_data)}, Dev data count: {len(dev_data)}")


In [12]:
task = 'asqp'
data = 'rest15'
data_to = 'rest16'
# LLMSS/ouputs/asqp/rest15/zero_asqp_rest15_1.txt
# LLMSS/outputs/asqp/rest15/zero_asqp_rest15_1.txt'
source_folder = f'/home/elicer/LLMSS/outputs/{task}/{data}'
destination_folder = f'/home/elicer/ABSA/data/{task}/{data_to}'
target_folder = f'/home/elicer/ABSA/data/{task}/{data}'

file_name = os.path.join(target_folder, 'train.txt')
_, targets = split_read(file_name)

Data read. Total count:  834


In [13]:
for i in range(1, 11):
    # 복사할 파일 이름과 새 파일 이름 설정
    file_name = f'zero_{task}_{data}_{i}.txt'
    new_file_name = f'zero_{task}_{data}_{i}_train.txt'
    
    # 원본 파일과 대상 파일의 전체 경로 생성
    source_path = os.path.join(source_folder, file_name)
    destination_path = os.path.join(destination_folder, new_file_name)

    _, inputs = split_read(source_path)
    merge_save(inputs, targets, destination_path, 1)
    
    print(f'File {file_name} was copied to {destination_folder} with the new name {new_file_name}.')

Data read. Total count:  834
Input count: 834
Expanded target count: 834
Merged data count: 834
Data saved. Total count: 834
File zero_asqp_rest15_1.txt was copied to /home/elicer/ABSA/data/asqp/rest16 with the new name zero_asqp_rest15_1_train.txt.
Data read. Total count:  834
Input count: 834
Expanded target count: 834
Merged data count: 834
Data saved. Total count: 834
File zero_asqp_rest15_2.txt was copied to /home/elicer/ABSA/data/asqp/rest16 with the new name zero_asqp_rest15_2_train.txt.
Data read. Total count:  834
Input count: 834
Expanded target count: 834
Merged data count: 834
Data saved. Total count: 834
File zero_asqp_rest15_3.txt was copied to /home/elicer/ABSA/data/asqp/rest16 with the new name zero_asqp_rest15_3_train.txt.
Data read. Total count:  834
Input count: 834
Expanded target count: 834
Merged data count: 834
Data saved. Total count: 834
File zero_asqp_rest15_4.txt was copied to /home/elicer/ABSA/data/asqp/rest16 with the new name zero_asqp_rest15_4_train.txt.


In [ ]:
task = 'asqp'
data = 'rest15'
mothod = 'dlo'

if mothod == 'mvp':
    method_path = 'top_5_post_data1.0'
elif mothod == 'dlo':
    method_path = 'top_1_post_data1.0'
# model load
model_path = f'/home/elicer/#backup/mvp_seo/outputs/{task}/{data}/{method_path}/final'

tokenizer = T5Tokenizer.from_pretrained(model_path)
tfm_model = MyT5ForConditionalGeneration.from_pretrained(model_path)
model = T5FineTuner(args, tfm_model, tokenizer)

model_path = f'/home/elicer/ABSA/outputs/{mothod}/{task}/{data}/{method_path}/final'
# save load
model.model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)